In [3]:
import cv2
import numpy as np
import dlib
from keras.models import load_model
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, UpSampling2D, Flatten, Dense, Reshape, Input
from tensorflow.keras.models import Model


2025-02-25 18:57:05.190281: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-25 18:57:05.361040: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740520625.423766   64130 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740520625.437279   64130 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-25 18:57:05.625368: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

### FASE TREINO

In [4]:
IMG_SIZE = 64

def load_images(folder):
    images = []
    for img_name in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,img_name))
        img = cv2.resize(img,(IMG_SIZE,IMG_SIZE))
        img = img.astype("float32")/255.0
        images.append(img)
    return np.array(images)

In [5]:
faces_a = load_images("brad_pit")
faces_b = load_images("jackman")

In [6]:
input_img = Input(shape=(IMG_SIZE,IMG_SIZE,3))

x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = Conv2D(64, (3, 3), activation='relu', padding='same',strides=(2,2))(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same',strides=(2,2))(x)
x = Flatten()(x)
encoded = Dense(128,activation="relu")(x)

x_a = Dense(8*8*128,activation="relu")(encoded)
x_a = Reshape((8,8,128))(x_a)
x_a = UpSampling2D((2,2))(x_a)
x_a = Conv2D(64, (3, 3), activation='relu', padding='same')(x_a)
x_a = UpSampling2D((2,2))(x_a)
x_a = Conv2D(3,(3,3),activation="relu",padding="same")(x_a)
decoded_a = UpSampling2D((2,2))(x_a)


2025-02-25 18:57:08.649779: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [7]:
x_b = Dense(8*8*128,activation="relu")(encoded)
x_b = Reshape((8,8,128))(x_b)
x_b = UpSampling2D((2,2))(x_b)
x_b = Conv2D(64, (3, 3), activation='relu', padding='same')(x_b)
x_b = UpSampling2D((2,2))(x_b)
x_b = Conv2D(3,(3,3),activation="relu",padding="same")(x_b)
decoded_b = UpSampling2D((2,2))(x_b)

In [8]:
autoencoder_a = Model(input_img,decoded_a)
autoencoder_b = Model(input_img,decoded_b)

autoencoder_a.compile(optimizer='adam', loss='mse')
autoencoder_b.compile(optimizer='adam', loss='mse')

autoencoder_a.fit(faces_a,faces_a,epochs=100,batch_size=16)
autoencoder_b.fit(faces_b,faces_b,epochs=100,batch_size=16)

encoder = Model(input_img,encoded)
decoder = Model(encoder.output,decoded_b)

autoencoder_b.summary()





Epoch 1/100
1/7 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - loss: 0.2470

2025-02-25 18:57:10.152435: E tensorflow/core/util/util.cc:131] oneDNN supports DT_INT32 only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - loss: 0.3652
Epoch 2/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.2528
Epoch 3/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.2076
Epoch 4/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.1779
Epoch 5/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.1480
Epoch 6/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.1225
Epoch 7/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.0862
Epoch 8/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.0820
Epoch 9/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0773
Epoch 10/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.0744
Epoch 11/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0720
Epoch 12/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.0703
Epoch 13/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0668
Epoch 14/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0628
Epoch 15/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0579
Epoch 16/100
7/7 ━━━━━━━━━━━━━━

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     4,194,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8192)           │     1,056,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_1 (Reshape)             │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_3 (UpSampling2D)  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 16, 16, 64)     │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_4 (UpSampling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 32, 32, 3)      │         1,731 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_5 (UpSampling2D)  │ (None, 64, 64, 3)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,259,915 (62.03 MB)

 Trainable params: 5,419,971 (20.68 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 10,839,944 (41.35 MB)

In [9]:
for layer_a,layer_encoder in zip(autoencoder_a.layers[:6],encoder.layers[:6]):
    layer_encoder.set_weights(layer_a.get_weights())



for layer_b,layer_decoder in zip(autoencoder_b.layers[6:],decoder.layers[1:]):
    layer_decoder.set_weights(layer_b.get_weights())
    
    

### FASE DEEP FAKE


In [10]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [11]:
def get_landmarks(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    if len(faces) == 0:
        return None, None
    
    for face in faces:
        landmarks = predictor(gray, face)
        points = np.array([[landmarks.part(n).x, landmarks.part(n).y] for n in range(68)])
    
        return points, face
    return None,None

In [12]:
def align_face_function(image,target_landmarks):
    source_landmarks,face_region = get_landmarks(image)

    if source_landmarks is None:
        return None,None
    

    tranform_matrix,_ = cv2.estimateAffinePartial2D(source_landmarks,target_landmarks)
    align_face =  cv2.warpAffine(image,tranform_matrix,(image.shape[1],image.shape[0]))
    
    return align_face,face_region

In [13]:
def swap_face(image,reference_image):
    target_landmarks,_ = get_landmarks(reference_image)
    if target_landmarks is None:
        return image
    
    align_face,face_region = align_face_function(image,target_landmarks)
    if align_face is None:
        return image
    
    

    align_face = align_face.astype("float32")/255.0
    align_face = cv2.resize(align_face,(64,64))
    align_face = np.expand_dims(align_face,axis=0)

    encoded_face = encoder.predict(align_face)

    fake_face = decoder.predict(encoded_face)
    fake_face = (fake_face[0]*255).astype("uint8")
    x,y,w,h = face_region.left(),face_region.top(),face_region.width(),face_region.height()
    fake_face = cv2.resize(fake_face,(w,h))

    image[y:y+h,x:x+w] = fake_face

    return image

In [14]:
image = cv2.imread(os.path.join("brad_pit","001_c04300ef.jpg"))
reference_image = cv2.imread(os.path.join("jackman","001_9adc92c2.jpg"))

image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
reference_image = cv2.cvtColor(reference_image, cv2.COLOR_BGR2RGB)

image_fake = swap_face(image, reference_image)

cv2.imwrite("imagem_fake.jpg", cv2.cvtColor(image_fake, cv2.COLOR_RGB2BGR))
cv2.imshow("DeepFake", cv2.cvtColor(image_fake, cv2.COLOR_RGB2BGR))
cv2.waitKey(0)
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
